## Pipelines compuestos

Hasta ahora hemos visto la utilidad de los <i>pipelines</i> y cómo es que podemos usarlos. Pero hemos creado pipelines bastante sencillos, ¿no crees?

Vamos a crear uno un poco más complicado, pero para eso vamos a necesitar un dataset un poco más complicado también:

In [ ]:
from utils import load_complex_data

dataset = load_complex_data()
dataset

Son 6 columnas, una de ellas es un <code>ID</code>, <code>job</code>, <code>marital</code> son categorías, <code>balance</code>, <code>age</code> y <code>loyalty</code> son numéricas y <code>subscribed</code>, la variable objetivo es categórica binaria. 

Vamos a preparar este dataset.

## <code>ColumnTransformer</code>

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

one_hot_encode_categories = ColumnTransformer([
    (
        'one_hot_encode_categories', # Nombre de la transformación
        OneHotEncoder(sparse_output=False), # Transformación a aplicar
        ["job", 'marital'] # Columnas involucradas
    )
])

Vamos a ver qué es lo que hace con nuestro dataset después de entrenarlo con <code>fit</code>:

In [ ]:
one_hot_encode_categories.fit(dataset)

transformed_dataset = one_hot_encode_categories.transform(dataset)
transformed_dataset

Uno puede acceder a los elementos de <code>ColumnTransformer</code> con el atributo <code>named_transformers_</code> y de ahí vamos a acceder al atributo <code>categories_</code> para recuperar los encabezados:

In [ ]:
cats = one_hot_encode_categories.named_transformers_['one_hot_encode_categories'].categories_

Podemos usar esta función que cree para ver esta matriz como un dataframe con las columnas:

In [ ]:
from utils import show_transformed_data

show_transformed_data(transformed_dataset, cats)

## Nested pipelines

Vamos a hacer algo con las variable <code>age</code>. Lo primero a notar es que la variable <code>age</code> tiene valores nulos, hay que imputar sus valores y después vamos a discretizarla, hagamos un pipeline para eso:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer

handle_age_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('discretize', KBinsDiscretizer(encode="onehot-dense"))
])

Si lo probamos pasando la columna <code>age</code>:

In [ ]:
handle_age_pipeline.fit_transform(dataset[['age']])

Vamos a envolver este pipeline en un column transformer para que funcione directamente con el dataframe:

In [ ]:
handle_age_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('discretize', KBinsDiscretizer(encode="onehot-dense"))
])

handle_age_transformer = ColumnTransformer([
    (
        'handle_age_transformer', # Nombre de la transformación
        handle_age_pipeline, # Transformación a aplicar
        ["age"] # Columnas involucradas
    )
])

Y podemos verificar que funciona:

In [ ]:
handle_age_transformer.fit_transform(dataset)

## Dejando variables sin transformar

Puedes utilizar la cadena <code>passthrough</code> para dejar variables pasar sin ninguna transformación:

In [ ]:
let_loyalty_pass_transformer = ColumnTransformer([
    (
        'leave_loyalty_alone',
        'passthrough',
        ['loyalty']
    )
])

let_loyalty_pass_transformer.fit_transform(dataset)

## <code>FeatureUnion</code> para juntar todo

Vamos a re-crear todo lo que acabamos de hacer arriba

In [ ]:
# Ya lo vimos más arriba
one_hot_encode_categories = ColumnTransformer([
    (
        'one_hot_encode_categories', # Nombre de la transformación
        OneHotEncoder(sparse_output=False), # Transformación a aplicar
        ["job", 'marital'] # Columnas involucradas
    )
])

# Ya lo vimos más arriba
handle_age_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('discretize', KBinsDiscretizer(encode="onehot-dense"))
])
handle_age_transformer = ColumnTransformer([
    (
        'handle_age_transformer', # Nombre de la transformación
        handle_age_pipeline, # Transformación a aplicar
        ["age"] # Columnas involucradas
    )
])

# Ya lo vimos más arriba
let_loyalty_pass_transformer = ColumnTransformer([
    (
        'leave_loyalty_alone',
        'passthrough',
        ['loyalty']
    )
])

# Este es nuevo
from sklearn.preprocessing import StandardScaler

scale_balance = ColumnTransformer([
    ('scale_balance', StandardScaler(), ['balance'])
])

Recuerda que gracias a <code>ColumnTransformer</code> cada uno de estos transformadores individualmente actúa sobre solamente unas cuantas columnas del dataset y descarta el resto. Pero en realidad lo que queremos es generar un dataset único.

Podemos utilizar la clase <code>FeaturUnion</code> para juntar nuestras características horizontalmente:

In [ ]:
from sklearn.pipeline import FeatureUnion

all_the_features = FeatureUnion([
    ('one_hot_encode_categories', one_hot_encode_categories),
    ('handle_age_transformer', handle_age_transformer),
    ('let_loyalty_pass_transformer', let_loyalty_pass_transformer),
    ('scale_balance', scale_balance)
])

Y si llamamos a <code>fit_transform</code> obtendremos un nuevo dataset ya transformado:

In [ ]:
transformed_dataset = all_the_features.fit_transform(dataset)
transformed_dataset

Este daataset tiene 22 columnas:

In [ ]:
transformed_dataset.shape

 15 de ellas provienen de las variables categóricas <code>job</code>, <code>marital</code>, 5 proviene de la columna <code>age</code> que binarizamos, y luego <code>balance</code> y <code>loyalty</code> son las dos restantes. Y bueno, en el proceso nos deshicimos de la columna <code>ID</code> que no nos sirve para nada en este caso.

## Entrenando un modelo

Para terminar, vamos a agregar un modelo de machine learning al final para que sea la joya de la corona y tengamos todo en un mismo lugar.

Lo primero, vamos a utilizar <code>clone</code> para crear copias sin entrenar de todo nuestro pipeline ya creado:

In [ ]:
from sklearn.base import clone

feature_transformer = clone(all_the_features)

Creamos el pipeline final:

In [ ]:
from sklearn.linear_model import LogisticRegression

inference_pipeline = Pipeline([
    ('featurize', feature_transformer),
    ('classifier', LogisticRegression()),
])

Para visualizar qué es lo que está sucediendo, puedes visualizarlo simplemente dejándolo solo en una celda:

In [ ]:
inference_pipeline

Ahora si, vamos a entrenarlo como cualquier otro estimador:

In [ ]:
inference_pipeline.fit(
    dataset,
    dataset['subscribed']
)

Y si creamos un nuevo ejemplo, podemos ejecutar predict sin ningún problema:

In [ ]:
import pandas as pd

nuevos_datos = pd.DataFrame([
    {
        "ID": 2432,
        "job": "technician",
        "marital": "single",
        "balance": 90,
        "age": 34,
        "loyalty": 0.5
    }
])

nuevos_datos

In [ ]:
inference_pipeline.predict(nuevos_datos)

Y ya está, ¡ahora lo único que debes almacenar y compartir es el objeto <code>inference_pipeline</code>!

## ¿Cuándo usarlos y cuando no?

Como puedes ver, los pipelines son muy útiles en muchos casos y ofrecen diversas ventajas. Sin embargo, hay situaciones en las que no son la mejor opción. A continuación, se presentan algunos consejos generales sobre cuándo usar o no usar pipelines:

### <b>Cuándo usar pipelines:</b>

 1. Procesamiento secuencial: Si tu flujo de trabajo de aprendizaje automático sigue una estructura secuencial, los pipelines son ideales para organizar y simplificar el proceso.

 1. Validación cruzada y ajuste de hiperparámetros: Los pipelines facilitan la validación cruzada y el ajuste de hiperparámetros, asegurando que las transformaciones de datos se apliquen de manera consistente y evitando problemas como la fuga de datos.

 1. Reproducibilidad y mantenibilidad: Si deseas mejorar la reproducibilidad y mantenibilidad de tu código, los pipelines son una excelente opción, ya que permiten encapsular todo el flujo de trabajo en una sola estructura.

 1. Colaboración en proyectos: Si estás trabajando en un equipo, los pipelines pueden facilitar la colaboración al proporcionar una representación clara y coherente de las diferentes etapas del proceso de aprendizaje automático.

### <b>Cuándo no usar pipelines:</b>

 1. Preprocesamiento complejo: Si tu conjunto de datos requiere de operaciones que no pueden representarse fácilmente como transformadores de scikit-learn, es posible que los pipelines no sean adecuados.

 1. Flujos de trabajo personalizados: Si requieres de hacer transformaciones que no se ajusten a la estructura secuencial de un pipeline de scikit-learn, es posible que debas manejar los pasos manualmente.

 1. Modelos fuera de scikit-learn: Si estás utilizando modelos o herramientas de aprendizaje automático de otras bibliotecas que no siguen la API de scikit-learn, es posible que no puedas usar un pipeline directamente.

 1. Si estás lidiando con enormes cantidades de datos: puede que a veces sea mejor llevar a cabo transformaciones de datos en otros lenguajes, como SQL para ahorrarnos tiempo.

En resumen, los pipelines de scikit-learn son una herramienta poderosa para muchos flujos de trabajo de aprendizaje automático, pero pueden no ser adecuados para todas las situaciones. Considera las necesidades específicas y las limitaciones de tu proyecto antes de decidir si un pipeline es la mejor opción.
